In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\antar\Desktop\GenAi Project-newsResearchTool\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load variables from .env
load_dotenv()

# Get API key from environment variable
api_key = os.getenv("GOOGLE_API_KEY")

# Initialize Gemini LLM via LangChain
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-pro-latest",  # Model name
    google_api_key=api_key,               # Loaded from .env
    temperature=0.1                        # Controls randomness
)

In [4]:
from langchain.document_loaders import UnstructuredURLLoader

loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [6]:
len(docs)

33

In [7]:
docs[0]

Document(page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nTrending Stocks\n\nIndusInd Bank INE095A01012, INDUSINDBK, 532187\n\nOla Electric INE0LXG01040, OLAELEC, 544225\n\nKalyan Jeweller INE303R01014, KALYANKJIL, 543278\n\nReliance INE002A01018, RELIANCE, 500325\n\nJio Financial INE758E01017, JIOFIN, 543940\n\n\n\nQuotes\n\nMutual Funds\n\nCommodities\n\nFutures & Options\n\nCurrency\n\nNews\n\nCryptocurrency\n\nForum\n\nNotices\n\nVideos\n\nGlossary\n\nAll\n\nHello, Login Hello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistMy AlertsMy MessagesPrice AlertsMy Profile My PROMy PortfolioMy WatchlistMy AlertsMy MessagesPrice AlertsLogoutLoans up to ₹50 LAKHSFixed DepositsCredit CardsLifetime Free Credit ScoreChat with UsDownload AppFollow us on:\n\nGo Ad-Free\n\nMy Alerts', metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'})

In [8]:
import faiss

from langchain_community.vectorstores import FAISS
vectorindex = FAISS.from_documents(docs, embeddings)

In [9]:
import pickle

file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex, f)


In [10]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [11]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain

In [12]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', google_api_key=SecretStr('**********'), temperature=0.1, client= genai.GenerativeModel(
   model_name='models/gemini-1.5-pro-latest',
   generation_config={}.
   safety_settings={}
))), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t tr

In [13]:
query = "what is the price of Tiago iCNG?"
import langchain
langchain.debug=True

chain({"question": query}, return_only_outputs=True)

c:\Users\antar\Desktop\GenAi Project-newsResearchTool\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nBusiness News,\n\nBudget 2025 News,\n\nSensex, and\n\nNifty updates. Obtain\n\nPerso

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:ChatGoogleGenerativeAI] [8.19s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh\n",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": []
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh\n",
            "response_metadata": {
              "prompt_feedback": {
                "block_reason": 0,
                "safety_ratings": []
              },
              "finish_reason": "STOP",
              "safety_ratings": []
            },
            "type": "a

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:ChatGoogleGenerativeAI] [65.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\n\nSOURCES: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": []
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\n\nSOURCES: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n",
         

{'answer': 'The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\n\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}